In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/MyDrive/CMPT733_Final_Project")
!ls

 covid-19.csv
 covid_19_data.csv
 covid_tweet
 covid_tweet.csv
 data_analysis_visulization
 Find_Other_Tags.ipynb
 ieeeset_extraction.ipynb
 JKU_repo_ETL.ipynb
 Kevin_Milestone_Presentation.m4a
 media_accounts.txt
 Media_Analysis.ipynb
 milestone_presentation.docx
 Milestone_with_SFU_ppt_template.pptx
'New Recording 3.m4a'
 preprocess_visualization_covid19_us_2.ipynb
 preprocess_visualization_covid19_us.ipynb
 project.ipynb
'Project Report.gdoc'
 Project_Timeline.xlsx
 project_twitter_gettingdata.ipynb
 Sentiment_Analysis_v1.ipynb
 Sentiment_Analysis_Visualization.ipynb
 SFU-Powerpoint-Template-Standard.pptx
 state-geocodes-v2016.xls
 Tag_Clustering.ipynb
 tweet_sentiment_added
 tweet_sentiment_added_noise_removed
 us_counties_covid19_daily.csv
 us_covid19_daily.csv
 us_states_covid19_daily.csv


In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [ ]:
files = [file for file in os.listdir("tweet_sentiment_added_noise_removed") if "tag_#" in file]
files

['tag_#coronavirus.csv',
 'tag_#coronaupdate.csv',
 'tag_#selfisolating.csv',
 'tag_#quarantine.csv',
 'tag_#wearamask.csv',
 'tag_#stayhomestaysafe.csv',
 'tag_#pneumonia.csv',
 'tag_#herdimmunity.csv',
 'tag_#wfh.csv',
 'tag_#masks4all.csv',
 'tag_#covid19.csv',
 'tag_#faceshield.csv',
 'tag_#sarscov2.csv',
 'tag_#frontlineheroes.csv',
 'tag_#washyourhands.csv',
 'tag_#covid-19.csv',
 'tag_#coronavaccine.csv',
 'tag_#flattenthecurve.csv',
 'tag_#covidvaccine.csv',
 'tag_#workfromhome.csv',
 'tag_#socialbubble.csv',
 'tag_#ppe.csv',
 'tag_#socialdistancing.csv',
 'tag_#lockdown.csv',
 'tag_#pandemic.csv']

In [ ]:
def extract_tags(tag, Text):
    tags = re.findall(r"(#\w+)", Text)
    tags = [x.lower() for x in tags]
    if tag in tags:
        tags.remove(tag)
    return tags

In [ ]:
frames = []
for file in files: 
    new_df = pd.DataFrame()
    df = pd.read_csv('./tweet_sentiment_added_noise_removed/' + file, lineterminator='\n')
    tag = re.sub('.csv', '', re.sub('tag_', '', file))
    new_df['date'] = df['date']
    new_df['Scraped_Tag'] = tag
    new_df['Other_Tags'] = df['Text'].apply(lambda text: extract_tags(tag, text))
    if 'LikeCount' in df.columns:
        new_df['LikeCount'] = df['LikeCount']
    else:
        new_df['LikeCount'] = df['LikeCount\r']
    new_df['compound'] = df['compound']
    frames.append(new_df)

result = pd.concat(frames)
result#.iloc[3,:]['Text']

,date,Scraped_Tag,Other_Tags,LikeCount,compound
0,2020-04-12,#coronavirus,[#fbr],8,0.1027
1,2020-04-12,#coronavirus,"[#songwriters, #songathon]",6,0.7263
2,2020-04-12,#coronavirus,[#japan],11,0.4767
3,2020-04-12,#coronavirus,"[#pandemic, #stroke]",12,-0.9260
4,2020-04-12,#coronavirus,[#covid2019],6,0.0000
...,...,...,...,...,...
116387,2021-02-12,#pandemic,"[#brexit, #msm, #eu, #globalist, #education]",5,-0.1531
116388,2021-02-12,#pandemic,"[#un, #illegals]",6,-0.1531
116389,2021-02-12,#pandemic,"[#nicolasturgeon, #scotland, #independence, #c...",5,-0.6808
116390,2021-02-12,#pandemic,[#testing],5,-0.3765


In [ ]:
def extract_tags_with_count(tag, rows):
    tags = []
    for l in rows['Text'].apply(lambda text: extract_tags(tag, text)).to_list():
        tags += l
    return Counter(tags)

In [ ]:
tags_analysis = pd.DataFrame(columns=['Scraped_Tag', 'Other_Tags_in_Pos_tweet', 'Other_Tags_in_Neg_tweet', 'Other_Tags_in_Neu_tweet'])

for file in files: 
    new_df = pd.DataFrame()
    df = pd.read_csv('./tweet_sentiment_added_noise_removed/' + file, lineterminator='\n')
    tag = re.sub('.csv', '', re.sub('tag_', '', file))

    tags_analysis = tags_analysis.append({'Scraped_Tag': tag,
                                          'Other_Tags_in_Pos_tweet': extract_tags_with_count(tag, df.loc[df['compound'] >= 0.05]), 
                                          'Other_Tags_in_Neg_tweet': extract_tags_with_count(tag, df.loc[df['compound'] <= -0.05]), 
                                          'Other_Tags_in_Neu_tweet': extract_tags_with_count(tag, df.loc[(df['compound'] > -0.05) & (df['compound'] < 0.05)]),
                                          }, ignore_index=True)

tags_analysis = tags_analysis.set_index('Scraped_Tag')
tags_analysis

,Other_Tags_in_Pos_tweet,Other_Tags_in_Neg_tweet,Other_Tags_in_Neu_tweet
Scraped_Tag,,,
#coronavirus,"{'#fbr': 41, '#songwriters': 3, '#songathon': ...","{'#pandemic': 1312, '#stroke': 6, '#covid19': ...","{'#covid2019': 50, '#arizona': 72, '#covid': 1..."
#coronaupdate,"{'#jabalpur': 1, '#healthresponseteam': 1, '#m...","{'#madhubani': 28, '#stayhome': 20, '#indiafig...","{'#belfast': 10, '#socialdistancing': 37, '#co..."
#selfisolating,"{'#nlwx': 1, '#happyeaster2020': 1, '#quaranti...","{'#eviegrace': 1, '#easter': 3, '#covid19austr...","{'#covid19': 199, '#selfiesunday': 1, '#borisj..."
#quarantine,"{'#throwback': 41, '#tb': 4, '#easter': 34, '#...","{'#teamrocky': 1, '#easter2020': 1, '#mentalhe...","{'#bettercallsaul': 3, '#newjourney': 1, '#van..."
#wearamask,"{'#weloveournhs': 1, '#clapforcarers': 7, '#ch...","{'#stayathome': 607, '#usehandsanitizer': 3, '...","{'#covid19': 2929, '#flattenthecurve': 81, '#h..."
#stayhomestaysafe,"{'#curfew': 3, '#navajonationcurfew': 1, '#doo...","{'#happyeaster': 2, '#teamsunstrum': 1, '#covi...","{'#crypto': 1, '#litecoinfam': 1, '#daredevil'..."
#pneumonia,"{'#coronavirus': 32, '#flu': 13, '#nhs': 5, '#...","{'#tories': 1, '#pensions': 1, '#benefits': 1,...","{'#infectiousdiseases': 8, '#covid19': 80, '#c..."
#herdimmunity,"{'#borisjohnson': 54, '#dominiccummings': 38, ...","{'#mortality': 2, '#coronavirus': 296, '#boris...","{'#nhsheroes': 1, '#coronacrisisuk': 2, '#brex..."
#wfh,"{'#lifeonwebex': 8, '#workfromhome': 1666, '#i...","{'#guiltypleasure': 1, '#coronavirus': 82, '#d...","{'#photography': 37, '#art': 52, '#design': 67..."


In [ ]:
tag_list = tags_analysis.index.to_list()

In [ ]:
def remove_existing_tag(counter, tag_list):
    for tag in tag_list:
        counter.pop(tag, None)
    return counter

In [ ]:
tags_analysis['Other_Tags_in_Pos_tweet'].apply(lambda c: remove_existing_tag(c, tag_list))
tags_analysis['Other_Tags_in_Neg_tweet'].apply(lambda c: remove_existing_tag(c, tag_list))
tags_analysis['Other_Tags_in_Neu_tweet'].apply(lambda c: remove_existing_tag(c, tag_list))
tags_analysis

,Other_Tags_in_Pos_tweet,Other_Tags_in_Neg_tweet,Other_Tags_in_Neu_tweet
Scraped_Tag,,,
#coronavirus,"{'#fbr': 41, '#songwriters': 3, '#songathon': ...","{'#stroke': 6, '#election2020': 97, '#antiasia...","{'#covid2019': 50, '#arizona': 72, '#covid': 1..."
#coronaupdate,"{'#jabalpur': 1, '#healthresponseteam': 1, '#m...","{'#madhubani': 28, '#stayhome': 20, '#indiafig...","{'#belfast': 10, '#covid19uk': 11, '#stayathom..."
#selfisolating,"{'#nlwx': 1, '#happyeaster2020': 1, '#quaranti...","{'#eviegrace': 1, '#easter': 3, '#covid19austr...","{'#selfiesunday': 1, '#borisjohnson': 60, '#nh..."
#quarantine,"{'#throwback': 41, '#tb': 4, '#easter': 34, '#...","{'#teamrocky': 1, '#easter2020': 1, '#mentalhe...","{'#bettercallsaul': 3, '#newjourney': 1, '#van..."
#wearamask,"{'#weloveournhs': 1, '#clapforcarers': 7, '#ch...","{'#stayathome': 607, '#usehandsanitizer': 3, '...","{'#happyeaster2020': 1, '#washyourpaws': 2, '#..."
#stayhomestaysafe,"{'#curfew': 3, '#navajonationcurfew': 1, '#doo...","{'#happyeaster': 2, '#teamsunstrum': 1, '#covi...","{'#crypto': 1, '#litecoinfam': 1, '#daredevil'..."
#pneumonia,"{'#flu': 13, '#nhs': 5, '#amazing': 1, '#nhsth...","{'#tories': 1, '#pensions': 1, '#benefits': 1,...","{'#infectiousdiseases': 8, '#measles': 3, '#de..."
#herdimmunity,"{'#borisjohnson': 54, '#dominiccummings': 38, ...","{'#mortality': 2, '#borisjohnson': 96, '#uk': ...","{'#nhsheroes': 1, '#coronacrisisuk': 2, '#brex..."
#wfh,"{'#lifeonwebex': 8, '#ibadanunrest': 1, '#lago...","{'#guiltypleasure': 1, '#disparities': 1, '#un...","{'#photography': 37, '#art': 52, '#design': 67..."


In [ ]:
tags_analysis.to_csv('other_tags.csv')